In [2]:
import time as tm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
import pyautogui

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless=new')   # Running Chrome in headless mode
chrome_options.add_argument('--no-sandbox')    # Necessary for running in some environments

# Automatically download and use the correct version of chromedriver
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=chrome_options)
def fix_login():
    try: 
        # button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="cancel-button-top"]')))
        # button.click()
        # tm.sleep(1)
        button_xpath = '//button[@class="cancel-button-top"]'
        button = driver.find_element(By.XPATH, button_xpath)
        button.click()
        print("Click done phần đăng nhập DONE.")
    except:
        pass
        # print('Nút Không tồn tại hoặc lỗi!')
def fix_modal_close_is_large():
    try:
        button_xpath = '//button[@class="modal-close is-large"]'
        button = driver.find_element(By.XPATH, button_xpath)
        button.click()
        print("Click done phần đề xuất máy DONE.")
    except:
        # print('Nút Không tồn tại hoặc lỗi!')
        pass
def find_link():
    url = f'https://cellphones.com.vn/hang-cu/dien-thoai.html'
    driver.get(url)

    # Đợi một khoảng thời gian để trang đã tải đầy đủ
    tm.sleep(3)

    # Đợi trang tải hoàn toàn
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[@class="button btn-show-more button__show-more-product"]')))

    # Nhấn nút "Xem thêm" cho đến khi không còn nút
    while True:
        try:
            # try: 
            #     button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="cancel-button-top"]')))
            #     button.click()
            #     tm.sleep(3)
            #     fix_login()
            #     fix_modal_close_is_large()
            # except:
            #     pass
            # try: 
            #     button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="modal-close is-large"]')))
            #     button.click()
            #     tm.sleep(3)
            #     fix_login()
            #     fix_modal_close_is_large()
            # except:
            #     pass
            tm.sleep(3)
            fix_login()
            fix_modal_close_is_large()
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="button btn-show-more button__show-more-product"]')))
            button.click()
            tm.sleep(3)  
            print("Click 'Xem thêm' DONE")
        except Exception as e:
            print("Không tìm thấy nút 'Xem thêm' hoặc đã tải hết sản phẩm")
            print(f"Chi tiết lỗi: {e}")
            break

    lst_xpath = '//div[@class="product-list-filter is-flex is-flex-wrap-wrap"]/div/div[1]/a'
    tags = driver.find_elements(By.XPATH, lst_xpath)
    print(f"Số lượng thẻ tìm thấy: {len(tags)}")
    new_tags = [new_tag.get_attribute('href') for new_tag in tags]
    return new_tags
def find_title():
    title_element = driver.find_element(By.TAG_NAME, 'h1')
    title = title_element.text.strip()
    print(f"Tiêu đề sản phẩm: {title}")
    return title
def find_price(tag_url):
    try:
        try:
            sale_price_element = driver.find_elements(By.XPATH, "//p[@class='tpt---sale-price']")
            try:
                original_price_element = driver.find_elements(By.XPATH, "//p[@class='tpt---price']")
            except:
                original_price_element = []
        except:
            print("Lỗi loại 1")
        try:
            sale_price_element1 = driver.find_element(By.XPATH, "//p[@class='product__price--show']")
            try:
                original_price_element1 = driver.find_element(By.XPATH, "//p[@class='product__price--through']")
            except:
                original_price_element1 = []
        except:
            print("Lỗi loại 2")
        
        if len(sale_price_element) > 0 and len(original_price_element) >= 0:
            return 0,sale_price_element, original_price_element
        else:
            return 1,sale_price_element1, original_price_element1
    except Exception as e:
        print(e)
def save_to_csv(data,feature_name):
    df = pd.DataFrame(data, columns=feature_name)
    df.to_csv("products_pandas1.csv", index=False, encoding='utf-8-sig')
def data_save_to_csv(data, feature_name):
    data_save_to_csv = []
    for datai in data:
        data_result = []
        for title in feature_name:
            if title in datai:
                data_result.append(datai[title])
            else:
                data_result.append("")
        data_save_to_csv.append(data_result)
    return data_save_to_csv
def price_processing(result,sale_price_element, original_price_element):
    if result == 0:
        try:
            if len(original_price_element) > 0:
                return sale_price_element[1].text.strip(), original_price_element[0].text.strip()
            else:
                return sale_price_element[1].text.strip(), ""
        except:
            print("Xử lý Giá FAIL!")
    else:
        try:
            if original_price_element is None:
                return sale_price_element.text.strip(), ""

            else:
                return sale_price_element.text.strip(), original_price_element.text.strip()

        except:
            print("Xử lý Giá FAIL!")
def defind_data(data_raw):
    data = {"Tên sản phẩm":data_raw[0], "Giá bán cũ":data_raw[1], "Giá mới":data_raw[2]}
    hardcore_details = data_raw[3]
    hardcore_titles = data_raw[4]
    for title,detail in zip(hardcore_titles,hardcore_details):
        data[title] = detail

    return data
def save_link_to_file(new_tags):
    import pickle
    # Lưu vào file
    with open('product_links.pkl', 'wb') as file:
        pickle.dump(new_tags, file)
def load_file_link_to_file(file_name = 'product_links.pkl'):
    import pickle
    with open(file_name, 'rb') as file:
        loaded_links = pickle.load(file)
    return loaded_links

def find_hardcore1():
    for i in range(0,10):
        fix_modal_close_is_large()
        fix_login()
        pyautogui.scroll(-300)
        tm.sleep(1)
        fix_modal_close_is_large()
        try:
            path = '//button[@class="button button__show-modal-technical my-3 is-flex is-justify-content-center"]'
            button = driver.find_element(By.XPATH, path)
            button.click()
            print('Click xem cấu hình DONEEEEEEEEEEEEEEEEEEEEEEEEEEE!')
            return 1
        except:
            if i == 9:
                return 0
            print('Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!')
def find_hardcore():
    try:
        path = '//div[@class="px-3 py-2 is-flex is-align-items-center is-justify-content-space-between"]/div'
        hardcore_details = driver.find_elements(By.XPATH, path)
        path = '//div[@class="px-3 py-2 is-flex is-align-items-center is-justify-content-space-between"]/p'
        hardcore_titles = driver.find_elements(By.XPATH, path)
        print(f"Found {len(hardcore_titles)} hardware titles, {len(hardcore_details)} details.")
        print('Lấy cấu hình DONEEEEEEEEEE!')
        hardcore_titles = [title.text.strip() for title in hardcore_titles]
        hardcore_details = [detail.text.strip() for detail in hardcore_details]
        return hardcore_titles,hardcore_details
    except:
        print('Lấy cấu hình FAILLLLLLLLLLLLLLLL!')


if __name__ == "__main__":

    #B1 
    # new_tags = find_link()
    # #Hai ham nay dung de luu link ve file pkl khoi load lai mat thoi gian, neu dung thi bo # la duoc
    # save_link_to_file(new_tags)
    new_tags = load_file_link_to_file()
    #B2
    # Lặp qua từng URL và tải trang
    data = []
    feature_name = ["Tên sản phẩm", "Giá bán cũ", "Giá mới"]
    count = 0
    for tag_url in new_tags:
        # if count == 100:
        #     break
        print('Link count: ', count)
        try:
            # Điều hướng đến từng URL sản phẩm
            driver.get(tag_url)
            tm.sleep(3)  # Đợi một khoảng thời gian ngắn để trang tải hoàn toàn
            fix_login()
            fix_modal_close_is_large()

            title = find_title()
            result,sale_price_element, original_price_element = find_price(tag_url)
            sale_price_element, original_price_element = price_processing(result,sale_price_element, original_price_element)
            print(sale_price_element)
            print(original_price_element)

            if find_hardcore1() == 0:
                driver.get(tag_url)
                tm.sleep(3)  # Đợi một khoảng thời gian ngắn để trang tải hoàn toàn
                fix_login()
                fix_modal_close_is_large()
                find_hardcore1()

            hardcore_titles,hardcore_details = find_hardcore()

            data_raw = [title, sale_price_element, original_price_element, hardcore_details ,hardcore_titles]
            data_done = defind_data(data_raw)

            for title in hardcore_titles:
                if title not in feature_name:
                    feature_name.append(title)
            data.append(data_done)
            print("Len data: ", len(data))
            print("******************************************************************************")
            count += 1
        except Exception as e:
            count += 1
            print(f"Đã xảy ra lỗi khi lấy thông tin từ {tag_url}: {str(e)}")
            continue
    data_save_to_csv = data_save_to_csv(data, feature_name)
    save_to_csv(data_save_to_csv,feature_name)
    driver.quit()


Link count:  0
Tiêu đề sản phẩm: iPhone 14 Pro Max 128GB - Cũ Trầy Xước
21.590.000đ
29.990.000đ
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click done phần đề xuất máy DONE.
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình DONEEEEEEEEEEEEEEEEEEEEEEEEEEE!
Found 35 hardware titles, 35 details.
Lấy cấu hình DONEEEEEEEEEE!
Len data:  1
******************************************************************************
Link count:  1
Tiêu đề sản phẩm: Samsung Galaxy Z Fold5 12GB 256GB - Đã Kích Hoạt
23.290.000đ
40.990.000đ
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL!
Click xem cấu hình FAILL